In [1]:
import boto3
from boto3.s3.transfer import TransferConfig
import os
import threading
import sys 
import pathlib

train_image_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/230OG_Orthomosaic_export_WedMar08181810258552.tif"
train_label_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Training/Tree_polygons_00_230OG.geojson"
train_aoi_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Training/border_polygon_00_230OG.geojson"
train = [train_image_uri, train_label_uri, train_aoi_uri]

val_image_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/230OG_Orthomosaic_export_WedMar08181810258552.tif"
val_label_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Training/Tree_polygons_00_230OG.geojson"
val_aoi_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Training/border_polygon_00_230OG.geojson"
val = [val_image_uri, val_label_uri, val_aoi_uri]
train_val_keys = ["image.tif", "label.json", "aoi.json"]

pred_image_dir = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic"
pred_aoi_uri = "/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Training/border_polygon_00_230OG.geojson"
pred = [pred_image_dir, pred_aoi_uri]
pred_keys = ["{}.tif", "aoi.json"]

In [5]:
bucket = "rvlightning-experiments"
folder = "moffit-testing"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()
                
def upload(s3, local, pref, remote):
    s3.upload_file(local, bucket, pref + "/" + remote)
    print(local, "-->", remote)
    
def upload_large(s3, local, pref, remote):
    config = TransferConfig(multipart_threshold=1024*25, max_concurrency=10,
                            multipart_chunksize=1024*25, use_threads=True)
    s3.upload_file(local, bucket, pref + "/" + remote,
                Config=config,
                Callback=ProgressPercentage(local))
    print(local, "->", remote)
    
def upload_large_directory(s3, local, pref):
    for i, file in enumerate(pathlib.Path(local).glob("*.tif")):
        file = file.as_posix()
        upload_large(s3, file, pref, str(i)+".tif")

In [6]:
s3 = boto3.client("s3")

# for t,k in zip(train, train_val_keys):
#     upload_large(s3, t, "giustina-test/train", k)
    
# for t,k in zip(train, train_val_keys):
#     upload_large(s3, t, "giustina-test/val", k)
    
upload_large_directory(s3, pred[0], "giustina-test/pred/tiled")
upload_large(s3, pred[1], "giustina-test/pred", "aoi.json")

/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-18-15_export_ThuFeb09235404582203.tif  28651520 / 28651520.0  (100.00%)/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-18-15_export_ThuFeb09235404582203.tif -> 0.tif
/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-7-9_export_ThuFeb09235404582203.tif  29414612 / 29414612.0  (100.00%)/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-7-9_export_ThuFeb09235404582203.tif -> 1.tif
/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-7-8_export_ThuFeb09235404582203.tif  31141054 / 31141054.0  (100.00%)/home/aerotract/NAS/main/Clients/Giustina/Winter2023/100_230OG/Data/ortho/Tiled/230OG_Orthomosaic/230OG_Orthomosaic-7-8

In [7]:
def list_s3_folder(bucket, p):
    s3 = boto3.resource("s3")
    bucket = s3.Bucket(bucket)
    for obj in bucket.objects.filter(Prefix=p):
        print("s3://" + obj.bucket_name + "/" + obj.key)

In [ ]:
list_s3_folder("rvlightning-experiments", "giustina-test/pred/tiled")